# File to build up augmented training via manipulating FTLEs during training
here standard training and FTLE suppression training is compared

to continue an example from MLE_master make sure to use the identical random seed


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
import numpy as np
import importlib
from plots.plots import set_plot_defaults
set_plot_defaults() # set the plot default formatting

# Juptyer magic: For export. Makes the plots size right for the screen 
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

%config InlineBackend.figure_formats = ['svg'] 


torch.backends.cudnn.deterministic = True
seed = np.random.randint(1,200)
seed = 60 #59
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
print(seed)
g = torch.Generator()
g.manual_seed(seed)


# Data preparation

In [ ]:
cross_entropy = False #this leads to squared loss in the training
output_dim = 2 #for model architecture later, but need it already for dataloader
data_noise = 0.05
num_points = 5000
plotlim = [-2, 2]
subfolder = 'LE_training' #all the files generated from this notebook get saved into this subfolder

import os
if not os.path.exists(subfolder):
    os.makedirs(subfolder)


label = 'vector' #MSE allows both scalar and vector output, cross_entropy only allows "scalar" output here


from models.training import create_dataloader
dataloader, dataloader_viz = create_dataloader('moons', noise = data_noise, cross_entropy = cross_entropy, num_points = num_points, plotlim = plotlim, random_state = seed, label = label, filename = subfolder + '/trainingset')


## Model dynamics

In [ ]:
#Import of the model dynamics that describe the neural ODE
#The dynamics are based on the torchdiffeq package, that implements ODE solvers in the pytorch setting
from models.neural_odes import NeuralODEvar

#for neural ODE based networks the network width is constant. In this example the input is 2 dimensional
hidden_dim, data_dim = 5, 2 

#T is the end time of the neural ODE evolution, time_steps are the amount of discretization steps for the ODE solver
T, time_steps = 10, 100 #
step_size = T/time_steps
num_params = 1 #the number of distinct parameters present in the interval. they are spread equidistant over the interval [0, T]. As there are 100 time_steps, the interval is divided into 10 parts, each of length 1 with 10 time_steps per subinterval.
layers_hidden = 1 #the amount of hidden layers in the vector field (these layers do not correspond to time discretization but to the expressivity of the vector field)

non_linearity = 'tanh' #'relu'
architecture = 'inside' #outside




## Training and generating level sets

In [ ]:
num_epochs = 60 #number of optimization runs in which the dataset is used for gradient decent

#####LE training parameters
le_reg = 10.
time_interval = torch.tensor([0., 2* T/5]) #compute LE only on the early subinterval
le_threshold = 0.05

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

anode_control = NeuralODEvar(device, data_dim, hidden_dim, output_dim = output_dim, non_linearity=non_linearity, 
                    architecture=architecture, T=T, time_steps=time_steps, num_params = num_params, layers_hidden = layers_hidden, cross_entropy=cross_entropy)


optimizer_anode_control = torch.optim.Adam(anode_control.parameters(), lr=1e-3) 

In [ ]:


torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
anode = NeuralODEvar(device, data_dim, hidden_dim, output_dim = output_dim, non_linearity=non_linearity, 
                    architecture=architecture, T=T, time_steps=time_steps, num_params = num_params, layers_hidden = layers_hidden, cross_entropy=cross_entropy)


optimizer_anode = torch.optim.Adam(anode.parameters(), lr=1e-3) 


In [ ]:

from numpy import mean
import torch.nn as nn

from FTLE import input_to_output
from FTLE import LEs
import torch.nn.functional as F
from plots.plots import plot_FTLEs

import models.training
importlib.reload(models.training) # Reload the module to ensure the latest changes are applied

from models.training import FTLE_Trainer


losses = {'mse': nn.MSELoss(), 
          'cross_entropy': nn.CrossEntropyLoss(), 
          'ell1': nn.SmoothL1Loss()
}



In [ ]:
from models.training import doublebackTrainer



trainer_anode_control = doublebackTrainer(anode_control, optimizer_anode_control, device, cross_entropy=cross_entropy, turnpike = False,
                        verbose = True) 
trainer_anode_control.train(dataloader, num_epochs)

In [ ]:


trainer_anode = FTLE_Trainer(anode, device = device, le_reg = le_reg, time_interval = time_interval, le_threshold = le_threshold) 
trainer_anode.train(dataloader, num_epochs)

In [ ]:
from FTLE import LEs

x_batch, y_batch = next(iter(dataloader_viz))
print(x_batch[0:5])
input = torch.tensor([0,1.])
le_summed = torch.zeros(0)
for input in x_batch:
    print('input ', input)
    le = LEs(input,anode)
    print('les ', le)
    print('le_max ', le[0])
    le_summed += abs(le[0])
    
    break


In [ ]:

import plots.plots
importlib.reload(plots.plots) # Reload the module to ensure the latest changes are applied
from plots.plots import classification_levelsets
from IPython.display import Image, HTML
import time

footnote = f'{le_reg = }, le_time_interval = {trainer_anode.time_interval}, le_threshold = {trainer_anode.le_threshold} {num_epochs = }, {cross_entropy = }, \n {num_params = },{time_steps = }, {output_dim = }, {hidden_dim = }, {data_noise = }, {seed = }'
footnote_control = f'le_reg = 0, {num_epochs = }, {cross_entropy = }, {num_params = },\n {time_steps = }, {output_dim = }, {hidden_dim = }, {data_noise = }, {seed = }'

num_levels = 6
        
fig_name_base = os.path.join(subfolder, 'levelsets')
classification_levelsets(anode, fig_name_base, footnote = footnote, num_levels = num_levels)
fig_name_base_control = os.path.join(subfolder, 'levelsets_control')
classification_levelsets(anode_control, fig_name_base_control, footnote = footnote_control,  num_levels = num_levels)


#If i do not add the time.time() to the image url, the image is not reloaded in the notebook because of caching
display(HTML(f"""
<div style="display:flex; gap:10px;">
    <img src="{fig_name_base + '.png'}?{time.time()}" width="400">
    <img src="{fig_name_base_control + '.png'}?{time.time()}" width="400">
</div>
"""))

plt.plot(trainer_anode.histories['epoch_loss_history'], label = 'loss_ges')
plt.xlim(0, len(trainer_anode.histories['epoch_loss_history']) - 1)
plt.ylim(0)
plt.xlabel('Iterations')
plt.ylabel('Loss')
# plt.show()

plt.plot(trainer_anode.histories['epoch_loss_le_history'], label = 'loss_le')
plt.plot(trainer_anode_control.histories['epoch_loss_history'], label = 'loss_control')
plt.legend()
plt.savefig(subfolder + '/losses.png')
plt.show()





In [ ]:
import plots.plots
importlib.reload(plots.plots) # Reload the module to ensure the latest changes are applied
from plots.plots import trajectory_gif_new


for X_viz, y_viz in dataloader_viz:
    trajectory_gif_new(anode, X_viz[0:50], y_viz[0:50], timesteps=time_steps, filename = subfolder + '/trajectory_new.gif', axlim = 8, dpi = 100)
    break



for X_viz, y_viz in dataloader_viz:
    trajectory_gif_new(anode_control, X_viz[0:50], y_viz[0:50], timesteps=time_steps, filename = subfolder + '/trajectory_new_control.gif', axlim = 8, dpi = 100)
    break



In [ ]:

display(HTML(f"""
<div style="display:flex; gap:10px;">
    <img src="{subfolder + "/trajectory_new" + str(time_steps - 1) + ".png"}?{time.time()}" width="400">
    <img src="{subfolder + "/trajectory_new_control" + str(time_steps - 1) + ".png"}?{time.time()}" width="400">
</div>
"""))

traj = Image(filename=subfolder + "/trajectory_new.gif")
display(traj)

traj = Image(filename=subfolder + "/trajectory_new_control.gif")
display(traj)


In [ ]:
from FTLE import LEs

#Example of how to use the LEs function to compute Lyapunov exponents
# Define inputs
input1 = torch.tensor([1, 0], dtype=torch.float32)
time_interval = torch.tensor([0, T], dtype=torch.float32)

les = LEs(input1, anode, time_interval=time_interval) #computes the Lyapunov exponents (l1>=l2>=...l_min) for the input1 over the time interval
print(les)


# Plot ANODE FTLEs

In [ ]:
from FTLE import LE_grid
le_density = 100 #the amount of points in the grid for the Lyapunov exponent computation
output_max, output_min = LE_grid(anode,x_amount = le_density)

In [ ]:
from FTLE import LE_grid
output_max_control, output_min_control = LE_grid(anode_control,x_amount = le_density)

In [ ]:
importlib.reload(plots.plots) # Reload the module to ensure the latest changes are applied
from plots.plots import plot_FTLEs

plot_FTLEs(output_max, filename = subfolder + '/MLE_max')
plot_FTLEs(output_max_control, filename = subfolder + '/MLE_max_control')

mFTLEs = Image(filename=subfolder + '/MLE_max.png', width = 400)
display(mFTLEs)

mFTLEs_control = Image(filename=subfolder + '/MLE_max_control.png', width = 400)
display(mFTLEs_control)

Evolution of the Finite Time Lyapunov Exponents over the whole interval

## Video 1: FTLE for each subinterval

In [ ]:
importlib.reload(plots.plots)

import os

from plots.plots import plot_points_from_traj, input_to_traj_and_color, plot_vectorfield
from plots.plots import create_gif_subintervals
import imageio
import io

le_density = 50
save_pngs = True #whether to save the individual png files used to create the gifs

filename = subfolder + '/FTLE_per_param'
create_gif_subintervals(anode, le_density=le_density, filename=filename, save_pngs=save_pngs)
LEevo_test = Image(filename=filename + ".gif")
display(LEevo_test)

filename = subfolder + '/FTLE_per_param_control'
create_gif_subintervals(anode_control, le_density=le_density, filename=filename, save_pngs=save_pngs)
LEevo_test_control = Image(filename=filename + ".gif")
display(LEevo_test_control)

in the autonomous case any interval of same length should give the same LE. So the above gif makes sense!

## Video 2: FTLE for shrinking interval

In [ ]:
importlib.reload(plots.plots) # Reload the module to ensure the latest changes are applied
from plots.plots import create_gif_shrinkingintervals


create_gif_shrinkingintervals(anode, le_density = le_density, filename=subfolder + '/shrinking_LE', save_pngs=save_pngs)
LEevo_test = Image(filename=subfolder + '/shrinking_LE.gif')
display(LEevo_test)

In [ ]:
create_gif_shrinkingintervals(anode_control, le_density = le_density, filename=subfolder + '/shrinking_LE_control', save_pngs=save_pngs)
LEevo_control = Image(filename=subfolder + '/shrinking_LE_control.gif')
display(LEevo_control)